In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Modelling
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score

import lightgbm as lgb

# Hyperparameter tuning
from hyperopt import fmin, rand, tpe, space_eval, STATUS_OK, Trials, hp
from hyperopt.pyll.stochastic import sample
# from bayes_opt import BayesianOptimization
# import optuna

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/retail-transaction-dataset/Retail_Transaction_Dataset.csv


I will be creating a simple model to predict whether a customer would spend in month, and using bayesian hyperparameter optimisation and display improvements of the model using this.

I will be predicting a 9 months and using the prior 3 months as training.

In [2]:
df = pd.read_csv("/kaggle/input/retail-transaction-dataset/Retail_Transaction_Dataset.csv")
display(df)

,CustomerID,ProductID,Quantity,Price,TransactionDate,PaymentMethod,StoreLocation,ProductCategory,DiscountApplied(%),TotalAmount
0,109318,C,7,80.079844,12/26/2023 12:32,Cash,"176 Andrew Cliffs\nBaileyfort, HI 93354",Books,18.677100,455.862764
1,993229,C,4,75.195229,8/5/2023 0:00,Cash,"11635 William Well Suite 809\nEast Kara, MT 19483",Home Decor,14.121365,258.306546
2,579675,A,8,31.528816,3/11/2024 18:51,Cash,"910 Mendez Ville Suite 909\nPort Lauraland, MO...",Books,15.943701,212.015651
3,799826,D,5,98.880218,10/27/2023 22:00,PayPal,"87522 Sharon Corners Suite 500\nLake Tammy, MO...",Books,6.686337,461.343769
4,121413,A,7,93.188512,12/22/2023 11:38,Cash,"0070 Michelle Island Suite 143\nHoland, VA 80142",Electronics,4.030096,626.030484
...,...,...,...,...,...,...,...,...,...,...
99995,726461,A,2,56.078258,7/17/2023 16:59,Credit Card,"3632 Darren Station Apt. 553\nEricaborough, RI...",Clothing,18.345145,91.581240
99996,328056,A,6,88.516406,5/30/2023 9:04,Credit Card,"821 Taylor Shoals\nEvansville, IL 70845",Electronics,3.995541,509.878179
99997,887304,B,4,72.385564,8/25/2023 7:59,Credit Card,"50653 Kara Lakes\nStephanieborough, RI 94492",Clothing,17.423979,239.092472
99998,326401,C,5,66.542239,2/5/2024 19:45,PayPal,"18756 Mcfarland Way Suite 866\nBarnettside, PR...",Electronics,14.345018,284.983717


### **EDA**
---

In [3]:
#checking for na values
df.isnull().sum(axis = 0)

CustomerID            0
ProductID             0
Quantity              0
Price                 0
TransactionDate       0
PaymentMethod         0
StoreLocation         0
ProductCategory       0
DiscountApplied(%)    0
TotalAmount           0
dtype: int64

In [4]:
#checking data types
df.dtypes

CustomerID              int64
ProductID              object
Quantity                int64
Price                 float64
TransactionDate        object
PaymentMethod          object
StoreLocation          object
ProductCategory        object
DiscountApplied(%)    float64
TotalAmount           float64
dtype: object

In [5]:
#converting transaction date to datetime format.
df["TransactionDate"] = pd.to_datetime(df["TransactionDate"])

In [6]:
df["TransactionDate"].dtype

dtype('<M8[ns]')

In [7]:
#checking min and max transaction date
print(f"max date: {df['TransactionDate'].max()}")
print(f"min date: {df['TransactionDate'].min()}")

max date: 2024-04-28 22:22:00
min date: 2023-04-29 22:27:00


### **Feature Engineering**
---

Grouping data for each month for each customer. Features to engineer include:
* Previous 2 months average spend
* Spend trend over previous 2 months (up or down)
* Average quantity of transactions over previous 2 months

In [8]:
#Grouping all data to month level

# creating a month field
df['Month_Year'] = (df['TransactionDate'].dt.strftime('%m') + "/" + df['TransactionDate'].dt.strftime('%Y'))
display(df['Month_Year'])

0        12/2023
1        08/2023
2        03/2024
3        10/2023
4        12/2023
          ...   
99995    07/2023
99996    05/2023
99997    08/2023
99998    02/2024
99999    02/2024
Name: Month_Year, Length: 100000, dtype: object

In [9]:
#creating average previous two month spend

# creating average previous two month transactions

#creating transactions in previous month

In [10]:
#creating spend trend

#creating transactions trend